In [1]:
import os
import numpy as np
import pyoorb as oo

### Initialize Oorb

In [2]:
# Initialize oorb
ephfile = os.path.join(os.getenv('OORB_DATA'), 'de405.dat')
oo.pyoorb.oorb_init(ephfile)

0

In [3]:
timeScales = {'UTC': 1, 'UT1': 2, 'TT': 3, 'TAI': 4}
elemType = {'CART': 1, 'COM': 2, 'KEP': 3, 'DEL': 4, 'EQX': 5}

### Read in some orbits.

In [4]:
# Set up some orbits
# orb is id, 6 elements, epoch_mjd, H, G, element type index
# keplerian appears to be element type index 3
# orbits = numpy.array([0.,1.,2.,3.,4.,5.,6.,5373.,1.,1.,3.])
o = np.loadtxt('test.des', dtype=([('objid', np.str_, 20), ('format', np.str_, 3),
                                   ('q', float), ('e', float), ('inc', float), ('Omega', float),
                                   ('argperi', float), ('tperi', float), ('H', float), ('epoch', float),
                                   ('index', int), ('npar', int), ('moid', float), ('compcode', np.str_, 10)]),
               skiprows=1)

orbits = np.zeros([len(o), 12], dtype=np.double, order='F')
orbits[:, 0] = np.arange(0, len(o), 1, dtype=float)
orbits[:, 1] = o['q']
orbits[:, 2] = o['e']
orbits[:, 3] = o['inc']
orbits[:, 4] = o['Omega']
orbits[:, 5] = o['argperi']
orbits[:, 6] = o['tperi']
orbits[:, 7] = np.zeros(len(o), float) + elemType['COM']
orbits[:, 8] = o['epoch']
orbits[:, 9] = np.zeros(len(o), float) + timeScales['TT']
orbits[:, 10] = o['H']
orbits[:, 11] = np.zeros(len(o)) + 0.15
for i in range(3, 6):
    orbits[:, i] = np.radians(orbits[:, i])

In [5]:
orbits

array([[  0.00000000e+00,   2.72680464e+00,   1.43389847e-01,
          1.11295242e-01,   1.69610904e+00,   5.11783260e+00,
          5.37715237e+04,   2.00000000e+00,   5.44660000e+04,
          3.00000000e+00,   1.75820000e+01,   1.50000000e-01],
       [  1.00000000e+00,   2.64978782e+00,   5.60540098e-02,
          1.36881230e-01,   2.99464483e+00,   2.50204097e+00,
          5.49679725e+04,   2.00000000e+00,   5.44660000e+04,
          3.00000000e+00,   2.05140000e+01,   1.50000000e-01],
       [  2.00000000e+00,   2.82327567e+00,   8.02216088e-02,
          2.03150687e-01,   1.70781814e+00,   5.79273522e+00,
          5.36066833e+04,   2.00000000e+00,   5.44660000e+04,
          3.00000000e+00,   1.83040000e+01,   1.50000000e-01]])

### Generate ephemerides

This does n-body propagation to the series of times specified.

In [6]:
offset = 30.
times = [orbits[0][8], orbits[0][8] + offset]
timescale = [timeScales['UTC']] * len(times)
ephem_dates = np.array(list(zip(times, timescale)), dtype=np.double, order='F')
ephs, err = oo.pyoorb.oorb_ephemeris(in_orbits=orbits,
                                     in_obscode='I11',
                                     in_date_ephems=ephem_dates)
if err != 0:
    print(err)

In [7]:
i = 0
start = ephs[i][0]
end = ephs[i][1]
print("@ %f" % (ephem_dates[0][0]), start)
print("@ %f" % (ephem_dates[1][0]), end)

@ 54466.000000 [  3.15354161e+00   1.82578901e+02   5.88067583e+00   2.36498517e+01
   5.44660000e+04   1.00000000e+00   1.04457019e-01  -3.41723209e-03
   1.62736035e+01   9.97366501e+01]
@ 54496.000000 [  2.77919808e+00   1.83595895e+02   6.69572318e+00   2.32687274e+01
   5.44960000e+04   1.00000000e+00  -3.43432547e-02   5.77223374e-02
   1.26202699e+01   1.29325279e+02]


### Transform orbital elements

In [8]:
type2 = 'CART'
print("Transforming from COM to %s" % (type2))
newElems, err = oo.pyoorb.oorb_element_transformation(in_orbits=orbits, in_element_type=elemType[type2])
if err != 0:
    print("error", err)
print("Input and output orbital elements (COM / CART)")
print("in ", orbits[i])
print("out", newElems[i])

Transforming from COM to CART
Input and output orbital elements (COM / CART)
in  [  0.00000000e+00   2.72680464e+00   1.43389847e-01   1.11295242e-01
   1.69610904e+00   5.11783260e+00   5.37715237e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
out [  0.00000000e+00  -3.30147843e+00   9.67774744e-01   3.52552490e-01
  -3.39925149e-03  -8.14684133e-03   4.90706192e-04   1.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]


In [9]:
print("Transforming back from %s to COM" % (type2))
newElems2, err = oo.pyoorb.oorb_element_transformation(in_orbits=newElems, in_element_type=elemType['COM'])
if err != 0:
    print("error", err)
print("Input and output orbital elements (CART / COM)")
print("in ", newElems[i])
print("out", newElems2[i])

Transforming back from CART to COM
Input and output orbital elements (CART / COM)
in  [  0.00000000e+00  -3.30147843e+00   9.67774744e-01   3.52552490e-01
  -3.39925149e-03  -8.14684133e-03   4.90706192e-04   1.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
out [  0.00000000e+00   2.72680464e+00   1.43389847e-01   1.11295242e-01
   1.69610904e+00   5.11783260e+00   5.37715237e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]


In [10]:
print('Differences in the initial/final COM orb elements')
diffs = newElems2 - orbits
print(diffs)

Differences in the initial/final COM orb elements
[[  0.00000000e+00  -1.73194792e-14   3.21964677e-15   2.35922393e-16
   -4.44089210e-16   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00  -2.13162821e-14   3.71924713e-15   1.66533454e-16
   -1.02140518e-14   5.63993297e-14   1.45519152e-11   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00  -2.57571742e-14   3.15025783e-15   1.66533454e-16
    0.00000000e+00   7.72715225e-14   2.18278728e-11   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]]


There can be larger differences in the tPeri, as this is degenerate (previous or next orbit?). 

### Propagate orbits

You can do both n-body and 2-body propagation (swap \_nb to \_2b). 
At the moment, oorb\_propagation\* crashes if you use anything other than CART (cartesian) elements. So transform first.

In [11]:
print("Propagating orbits by %d days" % (offset))
epoch_orig = orbits[i][8]
epoch_new = epoch_orig + offset
newEpoch = np.array([epoch_new, timeScales['TT']], dtype='double', order='F')
# Note that currently you have to use CART format elements.
elems = newElems
newOorbElems, err = oo.pyoorb.oorb_propagation_nb(in_orbits=elems, in_epoch=newEpoch)
if err != 0:
    print("error", err)
print("Input and output orbital elements (0 and %d days)" % (offset))
print("in ", orbits[i])
print("out", newOorbElems[i])

Propagating orbits by 30 days
Input and output orbital elements (0 and 30 days)
in  [  0.00000000e+00   2.72680464e+00   1.43389847e-01   1.11295242e-01
   1.69610904e+00   5.11783260e+00   5.37715237e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
out [  0.00000000e+00  -3.39281543e+00   7.20543871e-01   3.66133292e-01
  -2.68965290e-03  -8.32572656e-03   4.14522375e-04   1.00000000e+00
   5.44960000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]


In [12]:
print("Transforming new orbital elements to COM")
newElems3, err = oo.pyoorb.oorb_element_transformation(in_orbits=newOorbElems, in_element_type=elemType['COM'])
if err != 0:
    print("error", err)
print("Input and output orbital elements (CART/COM)")
print("in ", orbits[i])
print("out", newElems3[i])
print('epoch', epoch_new, newElems3[i][8])

Transforming new orbital elements to COM
Input and output orbital elements (CART/COM)
in  [  0.00000000e+00   2.72680464e+00   1.43389847e-01   1.11295242e-01
   1.69610904e+00   5.11783260e+00   5.37715237e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
out [  0.00000000e+00   2.72662741e+00   1.43413828e-01   1.11295174e-01
   1.69610744e+00   5.11761118e+00   5.37714875e+04   2.00000000e+00
   5.44960000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
epoch 54496.0 54496.0


In [13]:
# Generate ephemerides with these new elements, to compare.
ephs2, err = oo.pyoorb.oorb_ephemeris(in_orbits=newElems3, in_obscode='I11',
                                     in_date_ephems=ephem_dates)
for i in range(len(newElems3)):
    print('Object %d' % i)
    print(' Orbit: ', orbits[i] )
    diffs = ephs[i] - ephs2[i]
    #print("Difference in ephemerides from ephemeris generation vs propagation + ephemeris generation")
    #print(diffs)
    print("Just difference in mas in RA and then Dec")
    deltaRA = diffs[:,1] * 3600. * 1000.
    deltaDec = diffs[:,2] * 3600. * 1000.
    print(' ', deltaRA, deltaDec)

Object 0
 Orbit:  [  0.00000000e+00   2.72680464e+00   1.43389847e-01   1.11295242e-01
   1.69610904e+00   5.11783260e+00   5.37715237e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.75820000e+01   1.50000000e-01]
Just difference in mas in RA and then Dec
  [  2.54772203e-05   2.91606739e-05] [ -1.07913678e-05  -1.26618716e-05]
Object 1
 Orbit:  [  1.00000000e+00   2.64978782e+00   5.60540098e-02   1.36881230e-01
   2.99464483e+00   2.50204097e+00   5.49679725e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   2.05140000e+01   1.50000000e-01]
Just difference in mas in RA and then Dec
  [  2.53749022e-05   2.92629920e-05] [ -5.07753839e-06  -4.98161512e-06]
Object 2
 Orbit:  [  2.00000000e+00   2.82327567e+00   8.02216088e-02   2.03150687e-01
   1.70781814e+00   5.79273522e+00   5.36066833e+04   2.00000000e+00
   5.44660000e+04   3.00000000e+00   1.83040000e+01   1.50000000e-01]
Just difference in mas in RA and then Dec
  [  1.33013600e-06   1.33013600e-06] [ -4.47641

Note that the difference can change depending on the length of time in 'offset'. Needs a bit further investigation, but I suspect this is due to the timescale of integration hard-coded into the different routines (?). 